In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import datetime, timedelta, date
import xlrd
import glob
import warnings
from pulp import *
import pytz


warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

Custom function definitions

In [2]:
def ensure_datetime(series):
    """
    Ensure that the column is a datetime object
    """
    if not pd.api.types.is_datetime64_any_dtype(series):
        series= pd.to_datetime(series, errors='coerce')
    return series
def ensure_level_of_data(df):
    """
    Ensure that there is no duplication after pre-processing
    """
    if df.duplicated().any():
        print("DataFrame contains duplicates. Dropping duplicates.")
        df = df.drop_duplicates().reset_index(drop=True)
    return df

In [ ]:
#load the location variables from the file
with open('location_variables.txt', 'r') as f:
    for line in f:
        exec(line)

run_date = pd.to_datetime(run_date)
run_time_naive = pd.to_datetime(run_time_naive)
run_time = pd.to_datetime(run_time)

In [ ]:
### Reading the preprocessed files

stock =  pd.read_csv(preprocessed_path + 'stock.csv' )
stock = ensure_level_of_data(stock)
load_details_df =  pd.read_csv(preprocessed_path + 'load_details.csv' )
load_details_df = ensure_level_of_data(load_details_df)
obs_df =  pd.read_csv(preprocessed_path + 'obs_stock.csv' )
obs_df = ensure_level_of_data(obs_df)
open_sto = pd.read_csv(preprocessed_path + 'open_sto.csv' )
open_sto = ensure_level_of_data(open_sto)
inventory_policy = pd.read_csv(preprocessed_path + 'inventory_policy.csv' )
inventory_policy = ensure_level_of_data(inventory_policy)
open_so = pd.read_csv(preprocessed_path + 'open_so.csv' )
open_so = ensure_level_of_data(open_so)
uom_df = pd.read_csv(preprocessed_path + 'uom.csv' )
uom_df = ensure_level_of_data(uom_df)
production = pd.read_csv(preprocessed_path + 'planned_production.csv' )
production = ensure_level_of_data(production)
actual_production = pd.read_csv(preprocessed_path + 'actual_production.csv' )
actual_production = ensure_level_of_data(actual_production)
planned_loads_df = pd.read_csv(preprocessed_path + 'planned_loads.csv' )
planned_loads_df = ensure_level_of_data(planned_loads_df)
outbound_loads_df = pd.read_csv(preprocessed_path + 'outbound_loads.csv' )
outbound_loads_df = ensure_level_of_data(outbound_loads_df)
inbound_loads_df = pd.read_csv(preprocessed_path + 'inbound_loads.csv' )
inbound_loads_df = ensure_level_of_data(inbound_loads_df)
lcp_data = pd.read_csv(preprocessed_path + 'lcp_data.csv' )
lcp_data = ensure_level_of_data(lcp_data)
standard_weights = pd.read_csv(preprocessed_path + 'standard_weights.csv' )
standard_weights = ensure_level_of_data(standard_weights)

In [ ]:
### Creatign dictionaries for all units of conversion

pc_to_hl_dict = dict(zip(uom_df['material_sk'], uom_df['PC_HL']))
pc_to_pal_dict = dict(zip(uom_df['material_sk'], uom_df['PC_PAL']))
pal_weight_dict = dict(zip(uom_df['material_sk'], uom_df['pal_weight_kg']))


In [ ]:
cols_to_convert_to_dt = ['origin_slot_arrival','origin_slot_departure']
outbound_loads_df[cols_to_convert_to_dt] = outbound_loads_df[cols_to_convert_to_dt].apply(ensure_datetime)
cols_to_convert_to_dt = ['destination_slot_arrival','destination_slot_departure']
inbound_loads_df[cols_to_convert_to_dt] = inbound_loads_df[cols_to_convert_to_dt].apply(ensure_datetime)
cols_to_convert_to_dt = ['origin_slot_arrival','origin_slot_departure','destination_slot_arrival','destination_slot_departure']
load_details_df[cols_to_convert_to_dt] = load_details_df[cols_to_convert_to_dt].apply(ensure_datetime)
cols_to_convert_to_dt = ['start_inflow_ts','end_outflow_ts']
production[cols_to_convert_to_dt] = production[cols_to_convert_to_dt].apply(ensure_datetime)

outbound_loads_df.rename(columns={'origin_slot_arrival':'Slot Booked From', 'origin_slot_departure':'Slot Booked To'}, inplace=True)
inbound_loads_df.rename(columns={'destination_slot_arrival':'Slot Booked From', 'destination_slot_departure':'Slot Booked To'}, inplace=True)

In [ ]:
def calculate_stock_available_sr(load, stock, open_so, open_sto_in, open_sto_out, production, actual_production, inventory_policy, run_time, run_type):
    sku = load['material_sk']
    source = load['Source']
    slot_booking_time = load['Slot Booked From']
    priority_flag = load['Priority Flag']

    try:
        # Total Stock at hand for this SKU and source location, also depends on the refersh rate
        stock_at_hand = stock.loc[
            (stock['material_sk'] == sku) & (stock['Source'] == source), 'Opening_Stock'].values[0]
    
    except IndexError:
        stock_at_hand = 0


    # Outgoing Open SO - total outgoing quantity of this SKU from this source for run date
    outgoing_so = open_so[
        (open_so['material_sk'] == sku) &
        (open_so['Source'] == source) &
        (open_so['Delivery Date'] == run_time.normalize())
    ]['open_so_out(HL)'].sum()

    # Incoming Open STO - total incoming quantity of this SKU to this source between model run time and (1 hour before slot booking time or truck arrival time)
    incoming_sto = open_sto_in[
        (open_sto_in['material_sk'] == sku) &
        (open_sto_in['Destination'] == source) &
        (open_sto_in['Slot Booked From'] >= run_time) &
        (open_sto_in['Slot Booked From'] <= slot_booking_time - timedelta(hours=1))
    ]['Total Quantity HL'].sum()

    # Planned production only for now, might have to add actual production
    # total planned production of this SKU at this source which started after (3 hours before truck arrival time or slot booking time) and supposed to end after model run time
    planned_production = production[
        (production['material_sk'] == sku) &
        (production['plant_code'] == source) &
        (production['end_outflow_ts'] >= run_time) &
        (production['start_inflow_ts'] >= slot_booking_time - timedelta(hours=3))
    ]['Production_HL'].sum()

    try:
        # Actual production for the whole day for this SKU and destination -- destination var is not defined always leading to error
        #check with Rachana on this
        actual_prod = actual_production[
            (actual_production['material_sk'] == sku) &
            (actual_production['plant_code'] == destination)
        ]['Production_HL'].sum()
    except:
        actual_prod = 0

    # Outgoing STO with priority flag check, will have to validate and check
    # Outgoing STO - total outgoing quantity of this SKU from this source between model run time and slot booking time with priority flag less than or equal to the load priority flag
    outgoing_sto = open_sto_out[
        (open_sto_out['material_sk'] == sku) &
        (open_sto_out['Source'] == source) &
        (open_sto_out['Slot Booked From'] > run_time) &
        (open_sto_out['Slot Booked From'] < slot_booking_time) &
        (open_sto_out['Priority Flag'] <= priority_flag)
    ]['Total Quantity HL'].sum()

    try: 
        # Safety stock
        safety_stock = inventory_policy.loc[(inventory_policy['material_sk'] == sku)&(inventory_policy['Source']==source), 'safety_stock'].values[0]
    except IndexError:
        safety_stock = 0
    
    ## adding IF statement here so that for initial assignment, there is no Safety Stock, but for top-ups there are
    if run_type == 'INIT':
        safety_stock = 0
    elif run_type == 'TOP-UP':
        safety_stock = safety_stock

    # Calculate the stock available
    stock_available = (stock_at_hand - outgoing_so + incoming_sto + planned_production + actual_prod - outgoing_sto - safety_stock)
    
    return stock_available, stock_at_hand, planned_production, actual_prod, outgoing_so + outgoing_sto, incoming_sto, safety_stock

In [ ]:
def calculate_stock_available_dest(load, stock, open_so, open_sto_in, open_sto_out, actual_production, production, inventory_policy, run_time):
    sku = load['material_sk']
    destination = load['Destination']
    priority_flag = load['Priority Flag']

    try:
        # Stock at hand at the destination
        stock_at_hand = stock.loc[
            (stock['material_sk'] == sku) & (stock['Source'] == destination), 'Opening_Stock'].values[0]
    except IndexError:
        stock_at_hand = 0

    # Outgoing SO orders for the whole day
    outgoing_so = open_so[
        (open_so['material_sk'] == sku) &
        (open_so['Source'] == destination) &
        (open_so['Delivery Date'] == run_time.normalize())
    ]['open_so_out(HL)'].sum()

    # Incoming Open STO orders for the whole day
    incoming_sto = open_sto_in[
        (open_sto_in['material_sk'] == sku) &
        (open_sto_in['Destination'] == destination) &
        (open_sto_in['Slot Booked From'].dt.normalize() == run_time.normalize())
    ]['Total Quantity HL'].sum()

    # Outgoing STO orders for the whole day
    outgoing_sto = open_sto_out[
        (open_sto_out['material_sk'] == sku) &
        (open_sto_out['Source'] == destination) &
        (open_sto_out['Slot Booked From'].dt.normalize() == run_time.normalize())
    ]['Total Quantity HL'].sum()

    try:
        # Planned production for the whole day
        planned_production = production[
            (production['material_sk'] == sku) &
            (production['plant_code'] == destination) &
            (production['end_outflow_ts'].dt.normalize() == run_time.normalize())
        ]['Production_HL'].sum()
    except:
        planned_production = 0

    try:
        # Actual production for the whole day
        actual_prod = actual_production[
            (actual_production['material_sk'] == sku) &
            (actual_production['plant_code'] == destination)
        ]['Production_HL'].sum()
    except:
        actual_prod = 0

    # Calculate the stock available at the destination
    stock_available = (stock_at_hand - outgoing_so + incoming_sto + planned_production + actual_prod - outgoing_sto)

    try:
        # Maximum stock (inventory policy)
        max_stock = inventory_policy.loc[
            (inventory_policy['material_sk'] == sku) & 
            (inventory_policy['Source'] == destination), 
            'max_stock'
        ].values[0]
    except IndexError:
        max_stock = 0

    # Calculate the demand at the destination
    demand = max_stock - stock_available

    try:
        # safety stock (inventory policy)
        safety_stock = inventory_policy.loc[
            (inventory_policy['material_sk'] == sku) & 
            (inventory_policy['Source'] == destination), 
            'safety_stock'
        ].values[0]
    except IndexError:
        safety_stock = 0

    oos_qty = (safety_stock - stock_available) if (safety_stock - stock_available>=0) else 0
    oos_per = oos_qty / demand
    oos_per = oos_per if oos_per>0 else 0

    return stock_available, demand, oos_per

In [ ]:
def calculate_end_of_day_stock(stock, open_so, open_sto_out, production, actual_production, swaps_df, run_time):
    updated_stock = stock.copy()

    for idx, row in updated_stock.iterrows():
        sku = row['material_sk']
        source = row['Source']

        # Stock at hand
        stock_at_hand = row['Opening_Stock']

        # Outgoing SO for the whole day
        outgoing_so = open_so[
            (open_so['material_sk'] == sku) &
            (open_so['Source'] == source) &
            (open_so['Delivery Date'] == run_time.normalize())
        ]['open_so_out(HL)'].sum()

        # Updated incoming Open STO
        incoming_sto = swaps_df[
            (swaps_df['material_sk'] == sku) &
            (swaps_df['Destination'] == source) &
            (swaps_df['Suggested_deployment(HL)'] >0)
        ]['Suggested_deployment(HL)'].sum()

        incoming_og_sto = open_sto_out[
            (open_sto_out['material_sk'] == sku) &
            (open_sto_out['Destination']==source)&
            (open_sto_out['at_risk_flag']==False)
        ]['Total Quantity HL'].sum()

        # Updated outgoing Open STO
        outgoing_sto = swaps_df[
            (swaps_df['material_sk'] == sku) &
            (swaps_df['Source'] == source) &
            (swaps_df['Suggested_deployment(HL)'] >0)
        ]['Suggested_deployment(HL)'].sum()

        outgoing_og_sto = open_sto_out[
            (open_sto_out['material_sk'] == sku) &
            (open_sto_out['Source']==source)&
            (open_sto_out['at_risk_flag']==False)
        ]['Total Quantity HL'].sum()

        
        try:
            # Planned production for the whole day
            planned_production = production[
                (production['material_sk'] == sku) &
                (production['plant_code'] == source) &
                (production['Prod End'].dt.normalize() == run_time.normalize())
            ]['Production_HL'].sum()
        except:
            planned_production = 0

        # Calculate the stock at hand at the end of the day
        stock_at_hand_end_of_day = stock_at_hand - outgoing_so + incoming_sto + incoming_og_sto - outgoing_sto - outgoing_og_sto + planned_production

        # Update the stock dataframe
        updated_stock.at[idx, 'Closing_Stock'] = stock_at_hand_end_of_day

    return updated_stock


In [ ]:
def optimise_loads(data):
    problem = LpProblem('Load Exchanging', LpMaximize)

    # Decision Variable for Quantity
    data['Qty_LPVar_Name'] = 'qty_' + data['material_sk'].astype(str) + '_' + data['load_id'].astype(str)
    data['Qty_LPVar'] = data['Qty_LPVar_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Continuous"))

    # OBJECTIVE FUNCTION
    # shipment_value = lpSum((data['lcp_rank_1'] + (0.5 * data['%_OOS_1'] + 0.5 * data['%_At_Risk_1'])/100 + data['Priority Flag_1'] + (1 - data['Waiting_time'])) * data['Qty_LPVar'] * data['HL_weight'])
    shipment_value = lpSum((data['lcp_rank_1'] + 1 + (1 * data['%_OOS_1'] + 1 * data['%_At_Risk_1']) + data['Priority Flag_1']) * data['Qty_LPVar'])
    problem +=shipment_value

    # CONSTRAINT: Sum of all HL quantity recommendations for a material_sk should be less than total HL stock on hand at the source
    for grp_name, grp_df in data.groupby(['material_sk', 'Source']):
        problem += lpSum(grp_df['Qty_LPVar']) <= grp_df['Stock_on_hand_sr(HL)'].iloc[0]
    
    # CONSTRAINT: Sum of all HL quantity recommendations for a material_sk should be less than Demand at the Destination
    for grp_name, grp_df in data.groupby(['material_sk', 'Destination']):
        problem += lpSum(grp_df['Qty_LPVar']) <= grp_df['demand_at_dt(HL)'].iloc[0]

    # CONSTRAINT: Sum of all recommended weights for a load should be less than the weight left on the truck(load)
    for grp_name, grp_df in data.groupby(['load_id']):
        problem += lpSum(qty * conv for qty, conv in zip(grp_df['Qty_LPVar'], grp_df['HL_weight'])) <= grp_df['available_Weight'].iloc[0]

    # CONSTRAINT: Sum of all recommended pallets should be less than the pallet space left on the truck
    for grp_name, grp_df in data.groupby(['load_id']):
        problem += lpSum(qty * conv for qty, conv in zip(grp_df['Qty_LPVar'], grp_df['HL_PAL'])) <= grp_df['available_PAL'].iloc[0]
        
    problem.solve(PULP_CBC_CMD(maxSeconds = 2700, threads = None, msg = 0))

    print(LpStatus[problem.status])
    return problem

In [ ]:
def process_loads(main_outbound_df, main_inbound_df, main_load_details, stock, open_so, open_sto, production, actual_production, inventory_policy, lcp_data, load_details_df, run_time, result_path, tag):
    # ### For manual runs
    # main_outbound_df = outbound_loads_df_d0[outbound_loads_df_d0['Slot Booked From']>run_time]
    # main_inbound_df = inbound_loads_df_d0
    # main_load_details = load_details_df_d0[load_details_df_d0['origin_slot_arrival']>run_time].reset_index(drop=True)
    # # stock = updated_stock.copy()
    # run_time= run_time
    # tag = 'D0'

    # Initialize a dictionary to store the results
    kpi_results    = {}
    print('Optimizing for '+tag)


    # Sorting the outbound loads as per slot booked from and priority flag
    main_outbound_df = main_outbound_df.sort_values(['Priority Flag', 'Slot Booked From']).reset_index(drop=True)
    print('## Total Number of Loads: ', main_outbound_df['load_id'].nunique())
    kpi_results['Total number of loads'] = {
        'Value': main_outbound_df['load_id'].nunique(),
        'Percentage': 100
    }

    if main_outbound_df['load_id'].nunique() == 0:
        print('## No loads to process')
        swaps_df = pd.DataFrame()
        main_load_details = pd.DataFrame()
        if tag == 'D0':
            return swaps_df, main_load_details, stock
        else:
            return swaps_df, main_load_details

    # Merging the outbound loads with the Open STO loads (to get material level details)
    open_sto_out = pd.merge(main_outbound_df, open_sto, on=['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 'Priority Flag'], how='inner')
    open_sto_in = pd.merge(main_inbound_df, open_sto, on=['load_id', 'Source', 'RFRC_NUM12', 'movement_type', 'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 'Priority Flag'] ,how='inner')

    open_sto_out[['Stock_on_hand_sr(HL)', 'Stock_sr', 'Planned_production_sr', 'Actual_production_sr', 'Outgoing_SO_STO_sr', 'Incoming_STO_sr', 'Safety_Stock_sr']] = \
        open_sto_out.apply(lambda row: pd.Series(calculate_stock_available_sr(row, stock, open_so, open_sto_in, open_sto_out, production, actual_production, inventory_policy, run_time, 'INIT')), axis=1)

    open_sto_out['at_risk_flag'] = np.where(open_sto_out['Stock_on_hand_sr(HL)'] < open_sto_out['Total Quantity HL'], True, False)
    open_sto_out['Stock_on_hand_sr(HL)'] = np.where(open_sto_out['Stock_on_hand_sr(HL)'] < 0, 0, open_sto_out['Stock_on_hand_sr(HL)'])
    open_sto_out['qty_at_risk'] = np.where(open_sto_out['Total Quantity HL'] - open_sto_out['Stock_on_hand_sr(HL)'] <= 0, 0, open_sto_out['Total Quantity HL'] - open_sto_out['Stock_on_hand_sr(HL)'])
    open_sto_out['Total_feasible_order_qty'] = np.where(open_sto_out['at_risk_flag'] == True, open_sto_out['Total Quantity HL'] - open_sto_out['qty_at_risk'], open_sto_out['Total Quantity HL'])
    print('## Number of Loads that are risk due to insufficient stock: ', open_sto_out[open_sto_out['at_risk_flag'] == True]['load_id'].nunique())
    kpi_results['Number of Loads that are risk due to insufficient stock'] = {
        'Value': open_sto_out[open_sto_out['at_risk_flag'] == True]['load_id'].nunique(),
        'Percentage': open_sto_out[open_sto_out['at_risk_flag'] == True]['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }


    # Getting UOM for weights and for HL to PAL conversion
    open_sto_out['Total_feasible_order_qty_PC'] = open_sto_out['Total_feasible_order_qty'] / open_sto_out['material_sk'].map(pc_to_hl_dict)
    open_sto_out['Total_feasible_order_qty_PAL'] = open_sto_out['Total_feasible_order_qty_PC'] * open_sto_out['material_sk'].map(pc_to_pal_dict)
    open_sto_out['Total_feasible_order_qty_Weight'] = open_sto_out['Total_feasible_order_qty_PAL'] * open_sto_out['material_sk'].map(pal_weight_dict)

    ### Getting load level details of the original SKUs 
    new_load_data = open_sto_out.groupby(['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination', 'origin_shipping_location_sk',
        'destination_shipping_location_sk', 'Slot Booked From', 'Slot Booked To'], as_index=False).agg({'Total_feasible_order_qty_PAL': 'sum', 'Total_feasible_order_qty_Weight': 'sum'})

    ### Combining with the load level details
    main_load_details =pd.merge(main_load_details, new_load_data[['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination', 'Total_feasible_order_qty_PAL', 'Total_feasible_order_qty_Weight']],\
            on = ['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination'], how = 'left')

    # Combining the main_load_details with standard pallet and weight limit
    #Ask rachana what 26 represents here
    main_load_details['available_PAL'] = (26 - main_load_details['Total_feasible_order_qty_PAL']).astype(int)
    main_load_details['available_Weight'] = standard_weights.loc[standard_weights['Country'] == 'GB', 'weight_limit'].values[0] - (main_load_details['Total_feasible_order_qty_Weight']/1000)

    ### Getting the labels for each load:
    # Define a tolerance level for floating-point comparison
    tolerance = 1e-5

    # Initialize the 'Action' column with default empty strings
    main_load_details['Action'] = ''

    # Apply 'Load not at risk' condition
    #potential bug due to the abs condition which classifies loads with order req met as blank instead of 'Load not at risk', confirm with Rachana
    main_load_details.loc[
        (main_load_details['Original_Quantity_Ordered_PAL'] - main_load_details['Total_feasible_order_qty_PAL']).abs() < tolerance, 
        'Action'] = 'Load not at risk'

    # Apply 'At risk' condition
    main_load_details.loc[
        (main_load_details['Total_feasible_order_qty_PAL'] < main_load_details['Original_Quantity_Ordered_PAL']) & 
        (main_load_details['Action'] == ''),
        'Action'] = 'At risk'

    # Apply 'Light load' condition only if the other two are not satisfied
    main_load_details.loc[
        (main_load_details['available_PAL'] > 0) & 
        (main_load_details['available_Weight'] > 0) & 
        (main_load_details['Action'] == 'Load not at risk'), 
        'Action'] = 'Light load'

    ### new_load_data => only the light loads or the ones at risk
    new_load_data = main_load_details[main_load_details['Action'].isin(['Light load', 'At risk'])]
    new_load_data = new_load_data[(new_load_data['available_PAL']>0)&(new_load_data['available_Weight']>0)]
    print('## Loads that are at risk or have a light load: ', new_load_data['load_id'].nunique())
    kpi_results['Loads that are at risk or have a light load'] = {
        'Value': new_load_data['load_id'].nunique(),
        'Percentage': new_load_data['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }

    print('## Loads that are a light load: ', new_load_data[new_load_data['Action']=='Light load']['load_id'].nunique())
    kpi_results['Loads that are a light load'] = {
        'Value': new_load_data[new_load_data['Action']=='Light load']['load_id'].nunique(),
        'Percentage': new_load_data[new_load_data['Action']=='Light load']['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }

    if new_load_data['load_id'].nunique()==0:
        print('## No Loads at Risk')
        swaps_df = pd.DataFrame()
        main_load_details = pd.DataFrame()
        if tag == 'D0':
            return swaps_df, main_load_details, stock
        else:
            return swaps_df, main_load_details

    ## remove since this has been added to preprocessing
    # lcp_data.rename(columns={'origin_location_code':'Source', 'destination_location_code':'Destination', 'origin_plant_sk':'origin_shipping_location_sk', 'destination_plant_sk':'destination_shipping_location_sk'}, inplace=True)
    # lcp_data.drop(columns=['origin_shipping_location_sk', 'destination_shipping_location_sk'], inplace=True)


    ### Gotta add the origin and the destination sk
    data = pd.merge(new_load_data, lcp_data, on=['Source', 'Destination'], how='inner')
    data.rename(columns={'origin_slot_arrival':'Slot Booked From', 'origin_slot_departure':'Slot Booked To'}, inplace=True)


    data[['Stock_on_hand_sr(HL)', 'Stock_sr', 'Planned_production_sr', 'Actual_production_sr', 'Outgoing_SO_STO_sr', 'Incoming_STO_sr', 'Safety_Stock_sr']] = \
        data.apply(lambda row: pd.Series(calculate_stock_available_sr(row, stock, open_so, open_sto_in, open_sto_out, production, actual_production, inventory_policy, run_time, 'TOP-UP')), axis=1)
    data = data[data['Stock_on_hand_sr(HL)'] > 0]

    data[['Stock_on_hand_dt(HL)', 'demand_at_dt(HL)', '%_OOS']] = data.apply(lambda row: pd.Series(calculate_stock_available_dest(row, stock, open_so, open_sto_in, open_sto_out, production, actual_production, inventory_policy, run_time)), axis=1)
    data = data[(data['demand_at_dt(HL)'] > 0.0)&(data['Stock_on_hand_sr(HL)'] > 0.0)]


    print('## Loads that can have stock switched out using LCP data: ', data['load_id'].nunique())
    kpi_results['Loads that can have stock switched out using LCP data'] = {
        'Value':  data['load_id'].nunique(),
        'Percentage':  data['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }



    if data.shape[0]==0:
        print('## No possible replacement for the loads at risk')
        data.to_excel(f"{result_path}{tag}_pre_opti_model.xlsx", index=False)

        return None


    data = data[['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'Slot Booked From',
                    'Slot Booked To', 'Priority Flag', 'available_PAL', 'available_Weight',
                    'material_sk', 'material_code', 'lcp_rank', 'Stock_on_hand_sr(HL)', 'Stock_sr', 'Planned_production_sr', 'Actual_production_sr', 'Outgoing_SO_STO_sr', 'Incoming_STO_sr', 'Safety_Stock_sr', 'demand_at_dt(HL)', '%_OOS', 'Action']]
    data['HL_PAL'] = data['material_sk'].map(pc_to_pal_dict) / data['material_sk'].map(pc_to_hl_dict)
    data['HL_weight'] = data['material_sk'].map(pal_weight_dict) * data['HL_PAL'] / 1000

    data = pd.merge(data, obs_df[['material_code', 'Source', 'material_sk', '%_At_Risk']], on=['material_sk', 'material_code', 'Source'], how='left').fillna(0)

    data.drop_duplicates(subset=['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'Slot Booked From',
        'Slot Booked To', 'Priority Flag', 'material_sk', 'material_code'], inplace = True)



    # Operations to calculate the weights
    data['Waiting_time'] = data['Slot Booked From'] - run_time
    data['Waiting_time'] = data['Waiting_time'] / data['Waiting_time'].max()

    #### Change this: priority Flag?
    # Create a dictionary with the formula (15 - x) * 3 + 1 for Priority Flag values 0 to 16
    priority_mapping = {x: (15 - x) * 3 + 1 for x in range(17)}
    # Apply the dictionary to map the 'Priority Flag' column
    data['Priority Flag_1'] = data['Priority Flag'].map(priority_mapping)
    #bug here, %OOS returned is always <1, not multiplied by 100 in the function
    data['%_OOS_1'] = np.where(data['%_OOS'] >= 200, 200, data['%_OOS'])
    data['%_At_Risk_1'] = np.where(data['%_At_Risk'] >= 100, 100, data['%_At_Risk'])
    # Define the dictionary for mapping lcp_rank values
    lcp_rank_mapping = {
        1: 5,
        2: 4,  # You can assign 2 to other values if needed
        3: 3,
        4: 2,  # Example mapping for the remaining values
        5: 1
    }

    # Apply the dictionary to map the 'lcp_rank' column
    data['lcp_rank_1'] = data['lcp_rank'].map(lcp_rank_mapping)


    data['Weights'] = ((data['lcp_rank_1']) + (1 * data['%_OOS_1'] + 1 * data['%_At_Risk_1']) + (data['Priority Flag_1']))

    data.to_excel(f"{result_path}{tag}_pre_opti_model.xlsx", index=False)
    temp = new_load_data[new_load_data['load_id'].isin(data['load_id'].unique())]

    # Model run here
    problem = optimise_loads(data)

    # POST PROCESSING 

    data['Suggested_deployment(HL)'] = data['Qty_LPVar'].apply(lambda x: x.value())
    data['LP_Result_Status'] = LpStatus[problem.status]
    # data['Suggested_deployment(PAL)'] = data['Suggested_deployment(HL)'] * data['HL_PAL']
    # data['Suggested_deployment(Weight)'] = data['Suggested_deployment(HL)'] * data['HL_weight']
    results = data[data['Suggested_deployment(HL)'] > 0]
    results = results[['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'Priority Flag', \
        'material_sk', 'material_code', 'Stock_on_hand_sr(HL)', 'Stock_sr', 'Planned_production_sr', 'Actual_production_sr', 'Outgoing_SO_STO_sr', 'Incoming_STO_sr', 'Safety_Stock_sr'
        , 'Action', 'Suggested_deployment(HL)', 'demand_at_dt(HL)', '%_OOS']]
    

    # Swaps file creation
    swaps_df = open_sto_out[['load_id', 'RFRC_NUM12', 'movement_type', 'sales_document_item_code', 'Source', 'Destination', 'Priority Flag', 'material_sk', 'material_code',
                            'Stock_on_hand_sr(HL)', 'Stock_sr', 'Planned_production_sr', 'Actual_production_sr', 'Outgoing_SO_STO_sr', 'Incoming_STO_sr', 'Safety_Stock_sr', 'qty_at_risk', 'Total_feasible_order_qty', 'at_risk_flag']]
    swaps_df = swaps_df[swaps_df['at_risk_flag'] == True]

    swaps_df['Action'] = 'Swap-Out'
    swaps_df = swaps_df.rename(columns={'qty_at_risk': 'Swap_out_qty_HL', 'Total_feasible_order_qty': 'Suggested_deployment(HL)'})

    swaps_df = pd.concat([swaps_df, results])


    swaps_df['HL_PAL'] = swaps_df['material_sk'].map(pc_to_pal_dict) / swaps_df['material_sk'].map(pc_to_hl_dict)
    swaps_df['HL_weight'] = swaps_df['material_sk'].map(pal_weight_dict) * swaps_df['HL_PAL'] / 1000
    swaps_df['Suggested_deployment(PAL)'] = swaps_df['Suggested_deployment(HL)'] * swaps_df['HL_PAL']
    swaps_df['Suggested_deployment(Weight)'] = swaps_df['Suggested_deployment(HL)'] * swaps_df['HL_weight']
    swaps_df['Suggested_deployment(PC)'] = swaps_df['Suggested_deployment(PAL)'] / swaps_df['material_sk'].map(pc_to_pal_dict)
    swaps_df['Rounded_Suggested_deployment(PAL)'] = np.round(swaps_df['Suggested_deployment(PAL)'])
    swaps_df['Rounded_Suggested_deployment(HL)'] = swaps_df['Rounded_Suggested_deployment(PAL)'] /swaps_df['HL_PAL'] *1000
    swaps_df['Rounded_Suggested_deployment(Weight)'] = swaps_df['Rounded_Suggested_deployment(HL)'] * swaps_df['HL_weight']
    swaps_df['Rounded_Suggested_deployment(PC)'] = swaps_df['Rounded_Suggested_deployment(PAL)'] / swaps_df['material_sk'].map(pc_to_pal_dict)

    swapped_load_details = swaps_df.groupby(['load_id'], as_index = False).agg({'Rounded_Suggested_deployment(HL)':'sum', 'Rounded_Suggested_deployment(PAL)':'sum', 'Rounded_Suggested_deployment(Weight)':'sum'})
    main_load_details = pd.merge(main_load_details, swapped_load_details[['load_id', 'Rounded_Suggested_deployment(HL)', 'Rounded_Suggested_deployment(PAL)', 'Rounded_Suggested_deployment(Weight)']], on='load_id', how='left')
    main_load_details[['Rounded_Suggested_deployment(HL)', 'Rounded_Suggested_deployment(PAL)', 'Rounded_Suggested_deployment(Weight)']] = main_load_details[['Rounded_Suggested_deployment(HL)', 'Rounded_Suggested_deployment(PAL)', 'Rounded_Suggested_deployment(Weight)']].fillna(0)
    #bug : for overloaded trucks, total pal (26) - feasible order PAL is negative, need to check with Rachana
    main_load_details['Cancel_load'] = np.where((26-main_load_details['Total_feasible_order_qty_PAL']-main_load_details['Rounded_Suggested_deployment(PAL)'])/26 > 0.8, 'Yes', 'No')
    main_load_details['Rounded_Suggested_deployment(Weight)'] = main_load_details['Rounded_Suggested_deployment(Weight)'] / 1000

    main_load_details['Agreement to Recommendation(Yes/No)']= ''
    main_load_details['Recommendation Executed(Yes/No)']= ''
    main_load_details['Reason for non-agreement/non-execution']= ''

    main_load_details.drop_duplicates(subset=['load_id'], inplace=True)

    main_load_details = main_load_details[['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source',
        'Destination', 'origin_shipping_location_sk',
        'destination_shipping_location_sk', 'origin_slot_arrival',
        'origin_slot_departure', 'destination_slot_arrival',
        'destination_slot_departure', 'Action', 'Original_Quantity_Ordered_HL',
        'Original_Quantity_Ordered_PAL', 'Original_Quantity_Ordered_KG',
        'Total_feasible_order_qty_PAL', 'Total_feasible_order_qty_Weight',
        'available_PAL', 'available_Weight', 
        'Rounded_Suggested_deployment(HL)', 'Rounded_Suggested_deployment(PAL)',
        'Rounded_Suggested_deployment(Weight)', 'Cancel_load',
        'Agreement to Recommendation(Yes/No)',
        'Recommendation Executed(Yes/No)',
        'Reason for non-agreement/non-execution']]

    #bug : for overloaded trucks, total pal (26) - feasible order PAL is negative, need to check with Rachana
    main_load_details['available_PAL'] = np.where(main_load_details['available_PAL']<0, 0, main_load_details['available_PAL'])
    main_load_details['available_Weight'] = np.where(main_load_details['available_Weight']<0, 0, main_load_details['available_Weight'])

    main_load_details['Day_tag'] = tag
        
    # Generating output file 1 with the at-risk loads
    main_load_details.to_excel(f"{result_path}{tag}_load_level_report.xlsx", index=False)

    swaps_df = pd.merge(swaps_df, main_load_details[['RFRC_NUM12', 'origin_slot_arrival', 'origin_slot_departure', 'destination_slot_arrival', 'destination_slot_departure', 'Total_feasible_order_qty_PAL', 'Total_feasible_order_qty_Weight','origin_shipping_location_sk', 'destination_shipping_location_sk', 'Cancel_load']], on = ['RFRC_NUM12'], how= 'left')
    swaps_df['Swap_out_qty_HL'] = swaps_df['Swap_out_qty_HL'].fillna(0)
    # swaps_df = pd.merge(swaps_df, new_load_data.groupby(['load_id'], as_index=False)[['Total_feasible_order_qty_PAL', 'Total_feasible_order_qty_Weight']].sum(), on = ['load_id'], how = 'left')

    swaps_df['Action'] = np.where((swaps_df['Swap_out_qty_HL']>0)&(swaps_df['Suggested_deployment(HL)']>0), 'Swap-out (Update)', swaps_df['Action'])
    swaps_df['Action'] = np.where((swaps_df['Swap_out_qty_HL']>0)&(swaps_df['Suggested_deployment(HL)']==0), 'Swap-out (Delete)', swaps_df['Action'])

    swaps_df = pd.merge(swaps_df, open_sto_out[['load_id', 'material_sk', 'Total Quantity HL']], on = ['load_id', 'material_sk'], how = 'left').fillna(0)

    swaps_df['Action'] = np.where((swaps_df['Action']=='Light load')&(swaps_df['Total Quantity HL']==0), 'Top-up (New)', swaps_df['Action'])
    swaps_df['Action'] = np.where((swaps_df['Action']=='Light load')&(swaps_df['Total Quantity HL']!=0), 'Top-up (Update)', swaps_df['Action'])
    swaps_df['Action'] = np.where((swaps_df['Action']=='At risk'), 'Swap-in', swaps_df['Action'])


    swaps_df['Agreement to Recommendation(Yes/No)']= ''
    swaps_df['Recommendation Executed(Yes/No)']= ''
    swaps_df['Reason for non-agreement/non-execution']= ''

    swaps_df = swaps_df.fillna(0)

    swaps_df = swaps_df[['RFRC_NUM12', 'load_id', 'movement_type', 'sales_document_item_code', 'Priority Flag', 'origin_slot_arrival', 'origin_slot_departure', 'Source',
        'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 
        'material_sk', 'material_code', 'Action', 'Stock_on_hand_sr(HL)', 'Stock_sr', 'Planned_production_sr', 'Actual_production_sr', 'Outgoing_SO_STO_sr', 'Incoming_STO_sr', 'Safety_Stock_sr', 
        'demand_at_dt(HL)', '%_OOS', 'Total_feasible_order_qty_PAL',
        'Total_feasible_order_qty_Weight', 'Swap_out_qty_HL', 'Suggested_deployment(HL)',
        'Suggested_deployment(PAL)', 'Suggested_deployment(PC)',
        'Suggested_deployment(Weight)', 'Rounded_Suggested_deployment(HL)',
        'Rounded_Suggested_deployment(PAL)', 'Rounded_Suggested_deployment(PC)',
        'Rounded_Suggested_deployment(Weight)', 'Cancel_load',
        'Agreement to Recommendation(Yes/No)',
        'Recommendation Executed(Yes/No)',
        'Reason for non-agreement/non-execution']]

    swaps_df['Day_tag'] = tag
        
    swaps_df.rename(columns={'RFRC_NUM12':'STO Number'}, inplace=True)

    ### Dropping rows where the rounded recommendations for Swap-in or top-up are equal to zero
    swaps_df = swaps_df[(~swaps_df['Action'].isin(['Top-up (New)', 'Top-up (Update)', 'Swap-in']))|(swaps_df['Rounded_Suggested_deployment(PAL)']!=0)]


    # Writing the swaps file
    swaps_df.to_excel(f"{result_path}{tag}_Swaps.xlsx", index=False)
    
    print('### Number of loads that got swap-ins and top-ups: ', swaps_df[swaps_df['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique())
    kpi_results['Number of loads that got swap-ins and top-ups'] = {
        'Value':  swaps_df[swaps_df['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique(),
        'Percentage':  swaps_df[swaps_df['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }

    print('### Number of swap-ins and top-ups: ', swaps_df[swaps_df['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])].shape[0])
    kpi_results['Number of swap-ins and top-ups'] = {
        'Value':  swaps_df[swaps_df['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])].shape[0],
        'Percentage':  '-'
    }

    print('### Number of loads that got swap-ins: ', swaps_df[swaps_df['Action']=='Swap-in']['load_id'].nunique())
    kpi_results['Number of loads that got swap-ins'] = {
        'Value':  swaps_df[swaps_df['Action']=='Swap-in']['load_id'].nunique(),
        'Percentage':  swaps_df[swaps_df['Action']=='Swap-in']['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }
    print('### Number of swap-ins: ', swaps_df[swaps_df['Action']=='Swap-in'].shape[0])
    kpi_results['Number of swap-ins'] = {
        'Value':  swaps_df[swaps_df['Action']=='Swap-in'].shape[0],
        'Percentage':  '-'
    }

    ### initial volume:
    print('### Total initial Volume: ', main_load_details['Original_Quantity_Ordered_PAL'].sum())
    kpi_results['Total initial Volume in Pallets'] = {
        'Value':  main_load_details['Original_Quantity_Ordered_PAL'].sum(),
        'Percentage':  main_load_details['Original_Quantity_Ordered_PAL'].sum()/main_load_details['Original_Quantity_Ordered_PAL'].sum() * 100
    }
    ### Volumne at risk
    print('### Volume at risk: ', main_load_details['Original_Quantity_Ordered_PAL'].sum() - main_load_details['Total_feasible_order_qty_PAL'].sum())
    kpi_results['Volume at risk in Pallets'] = {
        'Value':  main_load_details['Original_Quantity_Ordered_PAL'].sum() - main_load_details['Total_feasible_order_qty_PAL'].sum(),
        'Percentage':  (main_load_details['Original_Quantity_Ordered_PAL'].sum() - main_load_details['Total_feasible_order_qty_PAL'].sum())/main_load_details['Original_Quantity_Ordered_PAL'].sum() * 100
    }
    ### Volume at risk that can be optimized for
    print('### Volume that can have stock switched in using LCP data: ', temp['Original_Quantity_Ordered_PAL'].sum()- temp['Total_feasible_order_qty_PAL'].sum())
    kpi_results['Volume that can have stock switched in using LCP data in Pallets'] = {
        'Value':  temp['Original_Quantity_Ordered_PAL'].sum()- temp['Total_feasible_order_qty_PAL'].sum(),
        'Percentage':  (temp['Original_Quantity_Ordered_PAL'].sum()- temp['Total_feasible_order_qty_PAL'].sum())/main_load_details['Original_Quantity_Ordered_PAL'].sum() * 100
    }
    ### Total optimised volume:
    print('### Total optimized volume: ', main_load_details['Rounded_Suggested_deployment(PAL)'].sum())
    kpi_results['Total optimized volume in Pallets'] = {
        'Value':  main_load_details['Rounded_Suggested_deployment(PAL)'].sum(),
        'Percentage':  (main_load_details['Rounded_Suggested_deployment(PAL)'].sum())/main_load_details['Original_Quantity_Ordered_PAL'].sum() * 100
    }
    print('### Total optimized volume of swap-ins and top-ups: ', swaps_df[swaps_df['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['Rounded_Suggested_deployment(PAL)'].sum())
    kpi_results['Total optimized volume of swap-ins and top-ups in Pallets'] = {
        'Value':  swaps_df[swaps_df['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['Rounded_Suggested_deployment(PAL)'].sum(),
        'Percentage':  (swaps_df[swaps_df['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['Rounded_Suggested_deployment(PAL)'].sum())/main_load_details['Original_Quantity_Ordered_PAL'].sum() * 100
    }
    print('### Total optimized volume of swap-ins: ', swaps_df[swaps_df['Action']=='Swap-in']['Rounded_Suggested_deployment(PAL)'].sum())
    kpi_results['Total optimized volume of swap-ins in Pallets'] = {
        'Value':  swaps_df[swaps_df['Action']=='Swap-in']['Rounded_Suggested_deployment(PAL)'].sum(),
        'Percentage':  (swaps_df[swaps_df['Action']=='Swap-in']['Rounded_Suggested_deployment(PAL)'].sum())/main_load_details['Original_Quantity_Ordered_PAL'].sum() * 100
    }

    # Convert KPI results into a DataFrame for saving as a table
    kpi_df = pd.DataFrame.from_dict(kpi_results, orient='index')

    # Save to excel file (or any other format)
    kpi_df.to_excel(f"{result_path}{tag}_Score_card.xlsx")

    if tag =='D1':
        return swaps_df, main_load_details
        
    ## Need to recheck the open_sto_in and open_sto_out part.
    updated_stock = calculate_end_of_day_stock(stock, open_so, open_sto_out, production, actual_production, swaps_df, run_time)

    return swaps_df, main_load_details, updated_stock

In [ ]:
### Getting the outbound and inbound STO loads for D0 and D+1
outbound_loads_df_d0 = outbound_loads_df[outbound_loads_df['Slot Booked From'].between(run_time.normalize(), run_time.normalize()+ timedelta(days=1) - timedelta(seconds= 1))]
outbound_loads_df_d1 = outbound_loads_df[outbound_loads_df['Slot Booked From'].between(run_time.normalize()+timedelta(days=1), run_time.normalize()+ timedelta(days=2))]

inbound_loads_df_d0 = inbound_loads_df[inbound_loads_df['Slot Booked From'].between(run_time.normalize(), run_time.normalize()+ timedelta(days=1) - timedelta(seconds= 1))]
inbound_loads_df_d1 = inbound_loads_df[inbound_loads_df['Slot Booked From'].between(run_time.normalize()+timedelta(days=1), run_time.normalize()+ timedelta(days=2))]

load_details_df_d0 = load_details_df[load_details_df['origin_slot_arrival'].between(run_time.normalize(), run_time.normalize()+ timedelta(days=1) - timedelta(seconds= 1))]
load_details_df_d1 = load_details_df[load_details_df['origin_slot_arrival'].between(run_time.normalize()+timedelta(days=1), run_time.normalize()+ timedelta(days=2))]


In [ ]:
### Running for D0
### Combining with the stock on hand with the outbound load orders  
## Filtering for future loads from the time of the run
main_outbound_df = outbound_loads_df_d0[outbound_loads_df_d0['Slot Booked From']>run_time]
main_inbound_df = inbound_loads_df_d0
main_load_details = load_details_df_d0[load_details_df_d0['origin_slot_arrival']>run_time].reset_index(drop=True)


swaps_df_d0, main_load_details_d0, updated_stock = process_loads(main_outbound_df, main_inbound_df, main_load_details, stock, open_so, open_sto, production, actual_production, inventory_policy, lcp_data, load_details_df, run_time, result_path, 'D0')

In [ ]:
swaps_df_d0

In [ ]:
# ### INitla volume:
# main_load_details_d0['Original_Quantity_Ordered_PAL'].sum()
# ### Volumne at risk
# main_load_details_d0['Original_Quantity_Ordered_PAL'].sum() - main_load_details_d0['Total_feasible_order_qty_PAL'].sum()
# ### Volume at risk that can be optimized for

# ### Total optimised volume:
# main_load_details_d0['Rounded_Suggested_deployment(PAL)'].sum()

In [ ]:
### Running for D1:
run_time_d1 = run_date +timedelta(days =1 )
main_outbound_df = outbound_loads_df_d1
main_inbound_df = inbound_loads_df_d1
main_load_details = load_details_df_d1[load_details_df_d1['origin_slot_arrival']>run_time_d1].reset_index(drop=True)


swaps_df_d1, main_load_details_d1 = process_loads(main_outbound_df, main_inbound_df, main_load_details, updated_stock, open_so, open_sto, production, actual_production, inventory_policy, lcp_data, load_details_df, run_time_d1, result_path, 'D1')

In [ ]:
print('Completed Running optimizer')